# Import packages

In [1]:
from keras.callbacks import Callback
from keras.layers import Dense, Input, Dropout, BatchNormalization, Activation
from keras.models import Model
from keras.optimizers import Adam, Nadam
from keras.utils.generic_utils import get_custom_objects
from sklearn.metrics import roc_auc_score

import keras
import keras.backend as K
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

import dataproc

np.random.seed(42) # NumPy
random.seed(42) # Python
tf.set_random_seed(42) # Tensorflow

Using TensorFlow backend.


# Data Loading

In [2]:
X_tr, X_val, X_test, y_tr, y_val, sub = dataproc.main()

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt', 'dist1',
       'dist2', 'C1', 'C2', 'C3', 'C4',
       ...
       '426', '427', '428', '429', '430', '431', '432', '433', '434', '435'],
      dtype='object', length=506)


# Modeling

In [3]:
# Compatible with tensorflow backend
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc_val: %s' % (str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
    
def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(K.epsilon()+pt_1))-K.mean((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0 + K.epsilon()))
    return focal_loss_fixed

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})
get_custom_objects().update({'focal_loss_fn': focal_loss()})

W1018 16:30:41.063925 19892 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [4]:
def create_model(loss_fn):
    inps = Input(shape=(X_tr.shape[1],))
    x = Dense(512, activation=custom_gelu)(inps)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation=custom_gelu)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inps, outputs=x)
    model.compile(
        optimizer=Nadam(),
        loss=[loss_fn]
    )
    #model.summary()
    return model

In [5]:
model_focal = create_model('focal_loss_fn')
model_bce = create_model('binary_crossentropy')

W1018 16:30:41.104925 19892 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1018 16:30:41.176961 19892 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1018 16:30:41.400161 19892 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1018 16:30:41.437377 19892 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:


In [6]:
model_bce.fit(
    X_tr, y_tr, epochs=8, batch_size=2048, validation_data=(X_val, y_val), verbose=True, 
    callbacks=[roc_callback(training_data=(X_val, y_tr), validation_data=(X_val, y_val))]
)
model_focal.fit(
    X_tr, y_tr, epochs=8, batch_size=2048, validation_data=(X_val, y_val), verbose=True, 
    callbacks=[roc_callback(training_data=(X_val, y_tr), validation_data=(X_val, y_val))]
)

Train on 472432 samples, validate on 118108 samples
Epoch 1/8
472432/472432 [==============================] - 55s 117us/step - loss: 0.2263 - val_loss: 0.1283
roc-auc_val: 0.8999                                                                                                    
Epoch 2/8
472432/472432 [==============================] - 53s 112us/step - loss: 0.1070 - val_loss: 0.0877
roc-auc_val: 0.9133                                                                                                    
Epoch 3/8
472432/472432 [==============================] - 52s 111us/step - loss: 0.0855 - val_loss: 0.0812
roc-auc_val: 0.9308                                                                                                    
Epoch 4/8
472432/472432 [==============================] - 53s 111us/step - loss: 0.0789 - val_loss: 0.0785
roc-auc_val: 0.9321                                                                                                    
Epoch 5/8
472432/472432 [===========

In [7]:
val_preds_bce = model_bce.predict(X_val).flatten()
val_preds_focal = model_focal.predict(X_val).flatten()

In [8]:
from scipy.stats import rankdata, spearmanr

print('BCE preds: ', roc_auc_score(y_val, val_preds_bce))
print('Focal preds: ',roc_auc_score(y_val, val_preds_focal))
print('Correlation matrix: ')
print(np.corrcoef(val_preds_bce, val_preds_focal))
print("Spearman's correlation: ", spearmanr(val_preds_bce, val_preds_focal).correlation)
print('Averaging: ', roc_auc_score(y_val, val_preds_bce + val_preds_focal))
print('Rank averaging: ', roc_auc_score(y_val, rankdata(val_preds_bce, method='dense') + rankdata(val_preds_focal, method='dense')))

BCE preds:  0.9541750763672989
Focal preds:  0.9470325728333012
Correlation matrix: 
[[1.        0.8336067]
 [0.8336067 1.       ]]
Spearman's correlation:  0.7964576326377572
Averaging:  0.9541475961143868
Rank averaging:  0.9553572562065871


# Predicting

In [9]:
model_bce.fit(X_val, y_val, epochs=2, batch_size=2048, verbose=True)
model_focal.fit(X_val, y_val, epochs=2, batch_size=2048, verbose=True)

Epoch 1/2
118108/118108 [==============================] - 10s 87us/step - loss: 0.0697
Epoch 2/2
118108/118108 [==============================] - 10s 87us/step - loss: 0.0607
Epoch 1/2
118108/118108 [==============================] - 11s 89us/step - loss: 0.0077
Epoch 2/2
118108/118108 [==============================] - 11s 94us/step - loss: 0.0071


In [15]:
sub.loc[:, 'isFraud'] = rankdata(model_bce.predict(X_test).flatten(), method='dense') + rankdata(model_focal.predict(X_test).flatten(), method='dense')
sub.loc[:, 'isFraud'] = sub.loc[:, 'isFraud']/sub.loc[:, 'isFraud'].max()
sub.to_csv('submission.csv')